In [1]:
import os
import sys
import argparse
import IPython 
from PIL import Image
from enum import Enum
from typing import Callable, List, Optional, Tuple, Union
from functools import partial

import torch
import torch.nn as nn
import torchvision
from torchvision.datasets import VisionDataset
from torchvision.transforms import transforms
import numpy as np
import pandas as pd
import skimage
from scipy import sparse
import matplotlib.pyplot as plt 
import torchxrayvision as xrv

from dinov2.data import SamplerType, make_data_loader, make_dataset
from dinov2.data.datasets import NIHChestXray, MC
from dinov2.data.transforms import make_xray_classification_eval_transform, make_classification_eval_transform
from dinov2.eval.setup import setup_and_build_model
from dinov2.eval.utils import ModelWithIntermediateLayers, ModelWithNormalize, evaluate, extract_features
from dinov2.utils import show_image_from_tensor

In [2]:
args = argparse.Namespace(config_file='dinov2/configs/eval/vits14_pretrain.yaml', pretrained_weights='models/dinov2_vits14_pretrain.pth', output_dir='results/NIH/dinov2_vits14/knn', opts=[], train_dataset_str='MC:split=TRAIN:root=/mnt/d/data/MC', val_dataset_str='MC:split=VAL:root=/mnt/d/data/MC', nb_knn=[5, 20, 50, 100, 200], temperature=0.07, gather_on_cpu=False, batch_size=8, n_per_class_list=[-1], n_tries=1, ngpus=1, nodes=1, timeout=2800, partition='learnlab', use_volta32=False, comment='', exclude='')
model, autocast_dtype = setup_and_build_model(args)
# model = ModelWithNormalize(model)

I20230820 02:50:39 8619 dinov2 config.py:60] git:
  sha: 8ed4cf18deedd673baeb8f013f7f3bec4956b4fd, status: has uncommitted changes, branch: main

I20230820 02:50:39 8619 dinov2 config.py:61] batch_size: 8
comment: 
config_file: dinov2/configs/eval/vits14_pretrain.yaml
exclude: 
gather_on_cpu: False
n_per_class_list: [-1]
n_tries: 1
nb_knn: [5, 20, 50, 100, 200]
ngpus: 1
nodes: 1
opts: ['train.output_dir=/mnt/c/Users/user/Desktop/dinov2/results/NIH/dinov2_vits14/knn']
output_dir: /mnt/c/Users/user/Desktop/dinov2/results/NIH/dinov2_vits14/knn
partition: learnlab
pretrained_weights: models/dinov2_vits14_pretrain.pth
temperature: 0.07
timeout: 2800
train_dataset_str: MC:split=TRAIN:root=/mnt/d/data/MC
use_volta32: False
val_dataset_str: MC:split=VAL:root=/mnt/d/data/MC
I20230820 02:50:39 8619 dinov2 config.py:27] sqrt scaling learning rate; base: 0.004, new: 0.001
I20230820 02:50:39 8619 dinov2 config.py:34] MODEL:
  WEIGHTS: ''
compute_precision:
  grad_scaler: true
  teacher:
    backbon

In [3]:
train_dataset_str = args.train_dataset_str
val_dataset_str = args.val_dataset_str
batch_size = args.batch_size
gather_on_cpu = args.gather_on_cpu
num_workers = 0

In [31]:
transform = make_classification_eval_transform(resize_size=448, crop_size=448)

train_dataset = make_dataset(
    dataset_str=args.train_dataset_str,
    transform=transform,
)
val_dataset = make_dataset(
    dataset_str=args.val_dataset_str,
    transform=transform,
)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1)

I20230820 03:00:19 8619 dinov2 loaders.py:90] using dataset: "MC:split=TRAIN:root=/mnt/d/data/MC"
I20230820 03:00:19 8619 dinov2 loaders.py:95] # of dataset samples: 138
I20230820 03:00:19 8619 dinov2 loaders.py:90] using dataset: "MC:split=VAL:root=/mnt/d/data/MC"
I20230820 03:00:19 8619 dinov2 loaders.py:95] # of dataset samples: 138


In [32]:
class LinearDecoder(torch.nn.Module):
    def __init__(self, in_channels, tokenW=32, tokenH=32, num_labels=1):
        super(LinearDecoder, self).__init__()

        self.in_channels = in_channels
        self.width = tokenW
        self.height = tokenH
        self.decoder = torch.nn.Conv2d(in_channels, num_labels, (1,1))

    def forward(self, embeddings):
        embeddings = embeddings.reshape(-1, self.height, self.width, self.in_channels)
        embeddings = embeddings.permute(0,3,1,2)

        return self.decoder(embeddings)

In [33]:
decoder = LinearDecoder(384).cuda()

In [37]:
for image, mask in train_loader:
    with torch.no_grad():
        features=model.forward_features(image.cuda())['x_norm_patchtokens']
    logits = decoder(features)
    logits = torch.nn.functional.interpolate(logits, size=448, mode="bilinear", align_corners=False)
    print(logits.shape)

KeyboardInterrupt: 